In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE  SCHEDULED_STOP
cluster-0016  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  4 08:08 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bucket3224' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith('.parquet'):
        paths.append(full_path + b.name)

In [ ]:
parquetFile = spark.read.parquet(*paths)
id_title_pairs = parquetFile.select("id", "title").rdd
data = id_title_pairs.collect() # Returns a list of Row(id=..., title=...)

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [9]:
# Count number of wiki pages
parquetFile.count()

6348910

In [ ]:
# Create a local SQLite database
db_filename = 'titles.db'
if os.path.exists(db_filename):
    os.remove(db_filename) # Clean up previous runs

print(f"Creating {db_filename}...")
conn = sqlite3.connect(db_filename)
c = conn.cursor()

# Create table with a primary key for fast lookups
c.execute('CREATE TABLE documents (id INTEGER PRIMARY KEY, title TEXT)')

In [ ]:
# Insert data efficiently
# We convert the Spark Rows to simple tuples (id, title) if necessary
data_tuples = [(row['id'], row['title']) for row in data]

print("Inserting data...")
c.executemany('INSERT INTO documents VALUES (?,?)', data_tuples)

conn.commit()
conn.close()
print(f"Database created successfully.")

In [ ]:
# Upload to Google Cloud Storage
blob_path = f'gs://{bucket_name}/{db_filename}'

print(f"Uploading to {blob_path}...")
!gsutil cp {db_filename} {blob_path}